## Ami-Br

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import (
    balanced_accuracy_score, roc_auc_score, confusion_matrix,
    classification_report, roc_curve
)
from huggingface_hub import login
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import pickle

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hugging Face login
login(token="your_huggingface_token_here")  # Replace with your Hugging Face token

# Load pretrained UNI model
uni_model = timm.create_model("hf-hub:MahmoodLab/uni", pretrained=True, init_values=1e-5, dynamic_img_size=True)
uni_model.eval().to(device)

# UNI transform
uni_config = resolve_data_config(uni_model.pretrained_cfg, model=uni_model)
transform = create_transform(**uni_config)

# Embedding extractor
def extract_embedding(img_path):
    image = Image.open(img_path).convert("RGB")
    tensor = transform(image).unsqueeze(0).to(device)
    with torch.inference_mode(), torch.autocast(device_type="cuda", dtype=torch.float16):
        output = uni_model(tensor)
    return output.squeeze(0).cpu()

# Dataset class
class InferenceDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.embeddings = [extract_embedding(p) for p in tqdm(image_paths, desc="Extracting embeddings")]
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Classifier head for UNI
class UNIBinaryClassifier(nn.Module):
    def __init__(self):
        super(UNIBinaryClassifier, self).__init__()
        self.classifier = nn.Linear(1024, 1)

    def forward(self, x):
        return self.classifier(x)

# Load test data
test_root = "/data/MELBA-AmiBr/Datasets_Stratified/AMi-Br/Test"
class_map = {"Atypical": 0, "Normal": 1}
class_names = ["Atypical", "Normal"]

image_paths, labels = [], []
for class_name, label_val in class_map.items():
    class_folder = os.path.join(test_root, class_name)
    for fname in os.listdir(class_folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.tif')):
            image_paths.append(os.path.join(class_folder, fname))
            labels.append(label_val)

# Prepare dataset and dataloader
test_dataset = InferenceDataset(image_paths, labels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8, pin_memory=True)

# Load fold models
num_folds = 5
model_paths = [f"uni_linear_probe_fold_{i + 1}_best.pth" for i in range(num_folds)]
models = []

for path in model_paths:
    model = UNIBinaryClassifier().to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()
    models.append(model)

# Evaluate each fold individually
true_labels = np.array(test_dataset.labels)
fold_bal_accs = []
fold_aurocs = []
fold_probs_dict = {}

for i, model in enumerate(models):
    fold_probs = []

    with torch.no_grad():
        for embeddings, _ in tqdm(test_loader, desc=f"Inference Fold {i + 1}"):
            embeddings = embeddings.to(device)
            logits = model(embeddings)
            probs = torch.sigmoid(logits).squeeze(1).cpu().numpy()
            fold_probs.extend(probs)

    fold_probs = np.array(fold_probs)
    fold_preds = (fold_probs > 0.5).astype(int)

    bal_acc = balanced_accuracy_score(true_labels, fold_preds)
    auroc = roc_auc_score(true_labels, fold_probs)

    fold_bal_accs.append(bal_acc)
    fold_aurocs.append(auroc)

    print(f"\nFold {i + 1} - Balanced Accuracy: {bal_acc:.4f}, AUROC: {auroc:.4f}")

    # Save predictions
    fold_probs_dict[f"fold_{i + 1}"] = {
        "probs": fold_probs,
        "preds": fold_preds,
        "true_labels": true_labels
    }

# Summary stats
mean_bal_acc = np.mean(fold_bal_accs)
std_bal_acc = np.std(fold_bal_accs)
mean_auroc = np.mean(fold_aurocs)
std_auroc = np.std(fold_aurocs)

print("\n--- Per-Fold Evaluation Summary (UNI) ---")
print(f"Balanced Accuracy: {mean_bal_acc:.4f} ± {std_bal_acc:.4f}")
print(f"AUROC: {mean_auroc:.4f} ± {std_auroc:.4f}")

# Save to disk
output_path = "uni_amibr_test_predictions.pkl"
with open(output_path, "wb") as f:
    pickle.dump(fold_probs_dict, f)

print(f"\nSaved fold predictions and labels to: {output_path}")


Extracting embeddings: 100%|██████████| 826/826 [00:15<00:00, 51.86it/s]
/tmp/ipykernel_2724318/1770436370.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_st


Fold 1 - Balanced Accuracy: 0.6405, AUROC: 0.7074


Inference Fold 2: 100%|██████████| 52/52 [00:00<00:00, 116.45it/s]



Fold 2 - Balanced Accuracy: 0.6411, AUROC: 0.7283


Inference Fold 3: 100%|██████████| 52/52 [00:00<00:00, 120.98it/s]



Fold 3 - Balanced Accuracy: 0.6616, AUROC: 0.7247


Inference Fold 4: 100%|██████████| 52/52 [00:00<00:00, 115.78it/s]



Fold 4 - Balanced Accuracy: 0.6356, AUROC: 0.7145


Inference Fold 5: 100%|██████████| 52/52 [00:00<00:00, 114.83it/s]


Fold 5 - Balanced Accuracy: 0.6598, AUROC: 0.7108

--- Per-Fold Evaluation Summary (UNI) ---
Balanced Accuracy: 0.6477 ± 0.0108
AUROC: 0.7171 ± 0.0080

Saved fold predictions and labels to: uni_amibr_test_predictions.pkl


## AtNorM-Br

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import (
    balanced_accuracy_score, roc_auc_score, confusion_matrix,
    classification_report, roc_curve
)
from huggingface_hub import login
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import pickle

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hugging Face login
login(token="your_huggingface_token_here")  # Replace with your Hugging Face token

# Load pretrained UNI model
uni_model = timm.create_model("hf-hub:MahmoodLab/uni", pretrained=True, init_values=1e-5, dynamic_img_size=True)
uni_model.eval().to(device)

# UNI transform
uni_config = resolve_data_config(uni_model.pretrained_cfg, model=uni_model)
transform = create_transform(**uni_config)

# Embedding extractor
def extract_embedding(img_path):
    image = Image.open(img_path).convert("RGB")
    tensor = transform(image).unsqueeze(0).to(device)
    with torch.inference_mode(), torch.autocast(device_type="cuda", dtype=torch.float16):
        output = uni_model(tensor)
    return output.squeeze(0).cpu()

# Dataset class
class InferenceDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.embeddings = [extract_embedding(p) for p in tqdm(image_paths, desc="Extracting embeddings")]
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Classifier head for UNI
class UNIBinaryClassifier(nn.Module):
    def __init__(self):
        super(UNIBinaryClassifier, self).__init__()
        self.classifier = nn.Linear(1024, 1)

    def forward(self, x):
        return self.classifier(x)

# Load test data
test_root = "/data/MELBA-AmiBr/Datasets_Stratified/AtNorM-Br"
class_map = {"Atypical": 0, "Normal": 1}
class_names = ["Atypical", "Normal"]

image_paths, labels = [], []
for class_name, label_val in class_map.items():
    class_folder = os.path.join(test_root, class_name)
    for fname in os.listdir(class_folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.tif')):
            image_paths.append(os.path.join(class_folder, fname))
            labels.append(label_val)

# Prepare dataset and dataloader
test_dataset = InferenceDataset(image_paths, labels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8, pin_memory=True)

# Load fold models
num_folds = 5
model_paths = [f"uni_linear_probe_fold_{i + 1}_best.pth" for i in range(num_folds)]
models = []

for path in model_paths:
    model = UNIBinaryClassifier().to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()
    models.append(model)

# Evaluate each fold individually
true_labels = np.array(test_dataset.labels)
fold_bal_accs = []
fold_aurocs = []
fold_probs_dict = {}

for i, model in enumerate(models):
    fold_probs = []

    with torch.no_grad():
        for embeddings, _ in tqdm(test_loader, desc=f"Inference Fold {i + 1}"):
            embeddings = embeddings.to(device)
            logits = model(embeddings)
            probs = torch.sigmoid(logits).squeeze(1).cpu().numpy()
            fold_probs.extend(probs)

    fold_probs = np.array(fold_probs)
    fold_preds = (fold_probs > 0.5).astype(int)

    bal_acc = balanced_accuracy_score(true_labels, fold_preds)
    auroc = roc_auc_score(true_labels, fold_probs)

    fold_bal_accs.append(bal_acc)
    fold_aurocs.append(auroc)

    print(f"\nFold {i + 1} - Balanced Accuracy: {bal_acc:.4f}, AUROC: {auroc:.4f}")

    # Save predictions
    fold_probs_dict[f"fold_{i + 1}"] = {
        "probs": fold_probs,
        "preds": fold_preds,
        "true_labels": true_labels
    }

# Summary stats
mean_bal_acc = np.mean(fold_bal_accs)
std_bal_acc = np.std(fold_bal_accs)
mean_auroc = np.mean(fold_aurocs)
std_auroc = np.std(fold_aurocs)

print("\n--- Per-Fold Evaluation Summary (UNI) ---")
print(f"Balanced Accuracy: {mean_bal_acc:.4f} ± {std_bal_acc:.4f}")
print(f"AUROC: {mean_auroc:.4f} ± {std_auroc:.4f}")

# Save to disk
output_path = "uni_atnorm-br_test_predictions.pkl"
with open(output_path, "wb") as f:
    pickle.dump(fold_probs_dict, f)

print(f"\nSaved fold predictions and labels to: {output_path}")


Extracting embeddings: 100%|██████████| 746/746 [00:14<00:00, 52.85it/s]
/tmp/ipykernel_2724318/10671313.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_stat


Fold 1 - Balanced Accuracy: 0.6193, AUROC: 0.6728


Inference Fold 2: 100%|██████████| 47/47 [00:00<00:00, 59.45it/s]



Fold 2 - Balanced Accuracy: 0.6232, AUROC: 0.6643


Inference Fold 3: 100%|██████████| 47/47 [00:00<00:00, 60.25it/s]



Fold 3 - Balanced Accuracy: 0.6208, AUROC: 0.6965


Inference Fold 4: 100%|██████████| 47/47 [00:00<00:00, 58.95it/s]



Fold 4 - Balanced Accuracy: 0.6233, AUROC: 0.6902


Inference Fold 5: 100%|██████████| 47/47 [00:00<00:00, 59.52it/s]


Fold 5 - Balanced Accuracy: 0.6222, AUROC: 0.6630

--- Per-Fold Evaluation Summary (UNI) ---
Balanced Accuracy: 0.6217 ± 0.0015
AUROC: 0.6774 ± 0.0136

Saved fold predictions and labels to: uni_atnorm-br_test_predictions.pkl


## AtNorM-MD

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import (
    balanced_accuracy_score, roc_auc_score, confusion_matrix,
    classification_report, roc_curve
)
from huggingface_hub import login
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import pickle

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hugging Face login
login(token="your_huggingface_token_here")  # Replace with your Hugging Face token

# Load pretrained UNI model
uni_model = timm.create_model("hf-hub:MahmoodLab/uni", pretrained=True, init_values=1e-5, dynamic_img_size=True)
uni_model.eval().to(device)

# UNI transform
uni_config = resolve_data_config(uni_model.pretrained_cfg, model=uni_model)
transform = create_transform(**uni_config)

# Embedding extractor
def extract_embedding(img_path):
    image = Image.open(img_path).convert("RGB")
    tensor = transform(image).unsqueeze(0).to(device)
    with torch.inference_mode(), torch.autocast(device_type="cuda", dtype=torch.float16):
        output = uni_model(tensor)
    return output.squeeze(0).cpu()

# Dataset class
class InferenceDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.embeddings = [extract_embedding(p) for p in tqdm(image_paths, desc="Extracting embeddings")]
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Classifier head for UNI
class UNIBinaryClassifier(nn.Module):
    def __init__(self):
        super(UNIBinaryClassifier, self).__init__()
        self.classifier = nn.Linear(1024, 1)

    def forward(self, x):
        return self.classifier(x)

# Load test data
test_root = "/data/MELBA-AmiBr/Datasets_Stratified/AtNorM-MD"
class_map = {"Atypical": 0, "Normal": 1}
class_names = ["Atypical", "Normal"]

image_paths, labels = [], []
for class_name, label_val in class_map.items():
    class_folder = os.path.join(test_root, class_name)
    for fname in os.listdir(class_folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.tif')):
            image_paths.append(os.path.join(class_folder, fname))
            labels.append(label_val)

# Prepare dataset and dataloader
test_dataset = InferenceDataset(image_paths, labels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8, pin_memory=True)

# Load fold models
num_folds = 5
model_paths = [f"uni_linear_probe_fold_{i + 1}_best.pth" for i in range(num_folds)]
models = []

for path in model_paths:
    model = UNIBinaryClassifier().to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()
    models.append(model)

# Evaluate each fold individually
true_labels = np.array(test_dataset.labels)
fold_bal_accs = []
fold_aurocs = []
fold_probs_dict = {}

for i, model in enumerate(models):
    fold_probs = []

    with torch.no_grad():
        for embeddings, _ in tqdm(test_loader, desc=f"Inference Fold {i + 1}"):
            embeddings = embeddings.to(device)
            logits = model(embeddings)
            probs = torch.sigmoid(logits).squeeze(1).cpu().numpy()
            fold_probs.extend(probs)

    fold_probs = np.array(fold_probs)
    fold_preds = (fold_probs > 0.5).astype(int)

    bal_acc = balanced_accuracy_score(true_labels, fold_preds)
    auroc = roc_auc_score(true_labels, fold_probs)

    fold_bal_accs.append(bal_acc)
    fold_aurocs.append(auroc)

    print(f"\nFold {i + 1} - Balanced Accuracy: {bal_acc:.4f}, AUROC: {auroc:.4f}")

    # Save predictions
    fold_probs_dict[f"fold_{i + 1}"] = {
        "probs": fold_probs,
        "preds": fold_preds,
        "true_labels": true_labels
    }

# Summary stats
mean_bal_acc = np.mean(fold_bal_accs)
std_bal_acc = np.std(fold_bal_accs)
mean_auroc = np.mean(fold_aurocs)
std_auroc = np.std(fold_aurocs)

print("\n--- Per-Fold Evaluation Summary (UNI) ---")
print(f"Balanced Accuracy: {mean_bal_acc:.4f} ± {std_bal_acc:.4f}")
print(f"AUROC: {mean_auroc:.4f} ± {std_auroc:.4f}")

# Save to disk
output_path = "uni_atnorm-md_test_predictions.pkl"
with open(output_path, "wb") as f:
    pickle.dump(fold_probs_dict, f)

print(f"\nSaved fold predictions and labels to: {output_path}")


Extracting embeddings: 100%|██████████| 2107/2107 [00:39<00:00, 53.74it/s]
/tmp/ipykernel_2724318/3364289630.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_


Fold 1 - Balanced Accuracy: 0.5730, AUROC: 0.6347


Inference Fold 2: 100%|██████████| 132/132 [00:01<00:00, 110.42it/s]



Fold 2 - Balanced Accuracy: 0.5927, AUROC: 0.6439


Inference Fold 3: 100%|██████████| 132/132 [00:01<00:00, 110.64it/s]



Fold 3 - Balanced Accuracy: 0.5594, AUROC: 0.6449


Inference Fold 4: 100%|██████████| 132/132 [00:01<00:00, 110.99it/s]



Fold 4 - Balanced Accuracy: 0.5823, AUROC: 0.6088


Inference Fold 5: 100%|██████████| 132/132 [00:01<00:00, 113.64it/s]


Fold 5 - Balanced Accuracy: 0.5502, AUROC: 0.5883

--- Per-Fold Evaluation Summary (UNI) ---
Balanced Accuracy: 0.5715 ± 0.0153
AUROC: 0.6241 ± 0.0222

Saved fold predictions and labels to: uni_atnorm-md_test_predictions.pkl
